In [1]:
# Imports

import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
# Get the Excel file and read

class ExcelReader(object):

    def __init__(self, path, ecID=1, vals=None):
        self.peers = None
        self.cstation = None
        self.load = None
        self.owner = None
        self.generator = None
        self.path = path
        self.ecID = ecID

        self.vals = vals

        # Placeholder dictionary to place the next ones
        self.parameterData = {}
        self.pD_network = {}

        # Resources
        self.resources = {}

        # General Info dictionary
        self.generalInfo = {}

        # Branch dictionary
        self.puSist = {}
        self.network = {}

        self.read_generalInfo()
        self.read_brachData()
        self.read_genData()
        self.read_loadData()
        self.read_storage()
        self.read_csData()
        self.read_peers()

        return


    def read_generalInfo(self):
        # General information sheet from excel
        generalInfo = pd.read_excel(self.path, sheet_name='General_Information', header=None)
        self.resources['numEC'] = generalInfo.values[2, 3]
        self.resources['period'] = generalInfo.values[3, 3]
        self.resources['periodDuration'] = generalInfo.values[4, 3]
        self.resources['owners'] = generalInfo.values[5, 3]

        # Read max imports and export quantities
        self.generalInfo['P_Max_Imp'] = generalInfo.values[23, 5:]
        self.generalInfo['P_Max_Exp'] = generalInfo.values[24, 5:]
        self.generalInfo['Energy_Buy_Price'] = generalInfo.values[25, 5:]
        self.generalInfo['Energy_SellPrice'] = generalInfo.values[26, 5:]
        self.generalInfo['GEE_CofA'] = generalInfo.values[27, 5:]
        self.generalInfo['GEE_CofB'] = generalInfo.values[28, 5:]
        self.generalInfo['GEE_CofC'] = generalInfo.values[29, 5:]

        return


    def read_brachData(self):
        # Branch data sheet from excel
        sheet_name_branch = 'Network_Info'
        data_branch = pd.read_excel(self.path, sheet_name=sheet_name_branch, header=None)

        # Set the values
        self.puSist['basePower'] = data_branch.values[1, 8]
        self.puSist['baseVoltage'] = data_branch.values[2, 8]
        self.puSist['baseCurrent'] = data_branch.values[3, 8]
        self.puSist['baseImpendance'] = data_branch.values[4, 8]

        busRef = data_branch.values[2, :3]
        busAux = np.unique(data_branch.values[22:, 1:3])

        voltageMax = np.zeros((max(busAux), self.resources['period']))
        voltageMin = np.zeros((max(busAux), self.resources['period']))
        angleMax = np.zeros((max(busAux), self.resources['period']))
        angleMin = np.zeros((max(busAux), self.resources['period']))

        for i in busAux:
            voltageMax[i-1, :] = data_branch.values[2, 5]
            voltageMin[i-1, :] = data_branch.values[2, 4]
            angleMax[i-1, :] = data_branch.values[4, 5]
            angleMin[i-1, :] = data_branch.values[4, 4]

        voltageMax[busRef[0]-1, :] = busRef[1]
        voltageMin[busRef[0]-1, :] = busRef[1]
        angleMax[busRef[0]-1, :] = busRef[2]
        angleMin[busRef[0]-1, :] = busRef[2]

        self.pD_network['voltageMax'] = voltageMax
        self.pD_network['voltageMin'] = voltageMin
        self.pD_network['angleMax'] = angleMax
        self.pD_network['angleMin'] = angleMin
        self.pD_network['busNum'] = np.reshape(busAux, (1, busAux.shape[0]))
        self.pD_network['busRef'] = busRef[0]

        # Conversion from data_branch
        numBranch = data_branch.values[:, 0].shape[0] - 22
        ybus = np.zeros((max(busAux), max(busAux)))
        diag = np.zeros((max(busAux), max(busAux)))
        branchID = np.zeros((max(busAux), max(busAux)))

        busI = data_branch.values[22:, 1]
        busJ = data_branch.values[22:, 2]

        # Admitance set
        for i in np.arange(busI.shape[0]):
            ybus[busI[i]-1, busJ[i]-1] = 1
            ybus[busJ[i]-1, busI[i]-1] = 1

            branchID[busI[i]-1, busJ[i]-1] = 1


        # Diagonal matrix values
        for i in np.arange(ybus.shape[0]):
            ybus[i, i] = 1
            diag[i, i] = 1

        branch = np.zeros((max(busAux), max(busAux), 4))
        idBusI = np.matlib.repmat(busI, 1, 4)
        idBusI = np.transpose(idBusI)

        idBusJ = np.matlib.repmat(busJ, 1, 4)
        idBusJ = np.transpose(idBusJ)

        # Branch and branchInfo
        mask = np.zeros(branch.shape)
        idBranchInfo = np.matlib.repmat(np.arange(1, 5), numBranch, 1)
        idBranchInfo = np.reshape(idBranchInfo.transpose(), (numBranch*4, 1))
        for i in np.arange(idBranchInfo.shape[0]):
            mask[idBusI[i, 0]-1, idBusJ[i, 0]-1, idBranchInfo[i, 0]-1] = 1

        branch[mask.astype(bool)] = data_branch.values[22:, 5:9].ravel()

        # Line data selection
        lineData = data_branch.values[1:, 0:9]
        lineData = pd.DataFrame(lineData)
        lineData = lineData.loc[(lineData.index != 17) & (lineData.index != 18)]
        for col in lineData.columns:
            lineData[col] = pd.to_numeric(lineData[col], errors='coerce')

        lineData.columns = ['Col{:02d}'.format(i) for i in np.arange(lineData.shape[1])]
        lineData = lineData[['Col00', 'Col01', 'Col02', 'Col05', 'Col06', 'Col07']]

        #Power Limit
        powerLimit = data_branch.values[1:, 0:9]
        powerLimit = pd.DataFrame(powerLimit)
        powerLimit = powerLimit.loc[(powerLimit.index != 17) & (powerLimit.index != 18)]
        for col in powerLimit.columns:
            powerLimit[col] = pd.to_numeric(powerLimit[col], errors='coerce')

        powerLimit.columns = ['Col{:02d}'.format(i) for i in np.arange(powerLimit.shape[1])]
        powerLimit = powerLimit[['Col00', 'Col01', 'Col02', 'Col08']]

        self.network['branch'] = branch
        self.network['lineData'] = lineData.values
        self.network['lineLimit'] = powerLimit
        self.pD_network['ybus'] = ybus
        self.pD_network['diag'] = diag
        self.pD_network['branch'] = branchID
        self.pD_network['numLine'] = numBranch
        return


    def _genOption(self):
        if self.vals is None:
            options = [1, 2, 3, 4, 5, 6, 7,
                       1, 2, 3, 4, 5, 6, 7]
        else:
            options = self.vals

        names = ['photovoltaic', 'wind', 'co-generation', 'biomass', 'waste-to-energy', 'small hydro',
                 'fuel cell', 'external supplier', 'none',
                 'Xup1', 'Xup2', 'Active Min', 'Active Max', 'Reactive Min', 'Reactive Max', 'Excess Max']

        idx = {}
        idx['PV'] = options[0]
        idx['Wind'] = options[1]
        idx['CHP'] = options[2]
        idx['Bio'] = options[3]
        idx['WtE'] = options[4]
        idx['SH'] = options[5]
        idx['FC'] = options[6]
        idx['ES'] = options[7]
        idx['No'] = options[8]

        return options, names, idx


    def read_genData(self):
        # Read the Excel
        sheet_name_gen = 'Generator_EC{}'.format(self.ecID)
        data_gen = pd.read_excel(self.path, sheet_name=sheet_name_gen, header=None)

        # Conversion from genData
        genOpt, genOptName, idx = self._genOption()

        # Find the column with the simulationPeriod
        idSimPer = np.where(data_gen.values[0, :] == self.resources['period'])[0][0]
        data_gen.drop(data_gen.columns[idSimPer+1:], axis=1, inplace=True)

        # Find the id number of Generators and Bus location
        idGenInfo = np.array([False for i in np.arange(idSimPer+1)])
        idGenInfo[:6] = True

        genInfo = data_gen.values[:, idGenInfo]
        for i in np.arange(genInfo.shape[0]):
            genInfo[i, :] = pd.to_numeric(genInfo[i, :], errors='coerce')

        numListGen1 = int(np.nanmax(genInfo[:, 1]))
        numListGen2 = int(np.nanmax(genInfo[:, genInfo.shape[1]-2]))
        numListGen = max(numListGen1, numListGen2)

        # Separate the information of each generator
        data_gen = pd.DataFrame(data_gen.values[:, idGenInfo==False])
        ids = pd.isnull(data_gen[data_gen.columns[0]])
        ids_values = np.where(ids == True)[0] + 1
        data_gen = data_gen.values[ids_values[0]:, :]

        ids = pd.isnull(data_gen[:, 0])
        data_gen = data_gen[ids==False, :]

        # Eliminate the NaN elements
        idNanElem = pd.isnull(genInfo[:, 1])
        idNanElemText = pd.isnull(genInfo[:, 0])
        genInfo = genInfo[idNanElem == False, :]
        genInfo = genInfo[:, [0, 1, 3]]

        # Determine the number of generators
        numGen = int(np.nanmax(genInfo[:, 0]))
        idGen = np.arange(1, numGen + 1)
        numListGen2 = int(data_gen.shape[0] / numGen)

        # Pass the excel values to the matlab structure
        generator = {}
        temp_limit = np.zeros((numGen, data_gen.shape[1], numListGen2))
        for i in np.arange(1, numGen+1):
            if (genInfo[int((i-1) * numListGen1 + 1), 2]) == 1:
                temp_limit[i-1, :, 0] = genInfo[int((i-1) * numListGen1 + 5), 2]
            else:
                temp_limit[i-1, :, 0] = data_gen[int((i-1) * numListGen2), :]

        for i in np.arange(2, numListGen2+1):
            temp_limit[:, :, i-1] = data_gen[(idGen-1) * numListGen2 + (i-1), :]
        generator['limit'] = temp_limit

        # Owner Info
        owner = {}
        temp_idx = np.array((idGen-1) * numListGen1 + 2, dtype=int)
        owner['gen'] = (np.array(genInfo[temp_idx, 2], dtype=int))

        # Generator Info
        infoGen = genInfo[:, 2]
        idx = pd.isnull(infoGen)
        infoGen[idx] = 0
        infoGen = np.array(infoGen, dtype=int)
        # Matlab uses Fortran-like indexing
        infoGen = np.reshape(infoGen, (numListGen1, numGen), order='F')
        generator['info'] = infoGen

        # Generator Location set
        genInBus = np.zeros((max(self.pD_network['busNum'].transpose())[0], numGen))
        idx = pd.isnull(genInfo[:, 0]) == False
        idx = np.where(idx == True)[0]

        idGenLoc = np.array(genInfo[idx, 2], dtype=int)
        idGenLoc = np.ravel_multi_index([idGenLoc.transpose(), idGen-1],
                                        dims=(genInBus.shape[0], genInBus.shape[1]),
                                        order='F') - 1

        temp_genInBus = genInBus.ravel()
        temp_genInBus[np.array(idGenLoc)] = 1
        temp_genInBus = np.reshape(temp_genInBus,
                                   (genInBus.shape[0], genInBus.shape[1]),
                                   order='F')
        resources = {'genInBus': temp_genInBus,
                     'numGen': numGen}

        self.owner = owner
        self.generator = generator
        self.parameterData['resources'] = resources

        return


    def _loadOption(self):

        options = None
        names = None
        if self.vals is not None:
            options = self.vals
        else:
            options = [1, 1, 2, 2, 3, 4, 5, 6, 7,
                       1, 2, 3, 4, 5, 6]

            names = ['Domestic 1', 'Domestic 2',
                     'Small Commerce 1', 'Small Commerce 2',
                      'Medium Commerce', 'Large Commerce',
                      'Medium Industrial', 'Large Industrial',
                      'None', 'Load Max', 'Red Max', 'Cut Max',
                      'Mov Max', 'In Mov Max', 'ENS Max']
        return options, names


    def _convertStrFlag(self, data, numFlag, initialFlag, opt, optName):

        res = np.zeros((data.shape[0], 1))
        for i in np.arange(numFlag):
            idStr = []
            for j in np.arange(data.shape[0]):
                idStr.append(str.lower(data[j]) == str.lower(optName[i + initialFlag]))
            res[idStr, 0] = opt[i + initialFlag]

        idEmpty = []
        for i in np.arange(data.shape[0]):
            idEmpty.append(str.lower(data[i]) == '')
        res[idEmpty, 0] = opt[numFlag + initialFlag]

        return res.ravel()


    def read_loadData(self):

        # Read the Excel
        sheet_name_load = 'Load_EC{}'.format(self.ecID)
        data_load = pd.read_excel(self.path, sheet_name=sheet_name_load, header=None)

        loadOpt, loadOptName = self._loadOption()

        # Find the column with the simulation Period
        idSimPer = np.where(data_load.values[0, :] == self.resources['period'])[0][0]

        # Copy and turn data_load DataFrame to numeric
        loadNumeric = data_load.copy(deep=True).to_numpy()
        for i in np.arange(loadNumeric.shape[0]):
            loadNumeric[i, :] = pd.to_numeric(loadNumeric[i, :], errors='coerce')

        loadText = data_load.to_numpy(copy=True)
        loadText[pd.notna(loadNumeric)] = np.nan
        loadText = np.reshape(loadText, (loadNumeric.shape[0],
                                         loadNumeric.shape[1]))

        # Find the ID number of generators and bus location
        idLoadInfo = pd.isnull(loadNumeric[0, :])
        loadInfo = loadNumeric[:, idLoadInfo]

        # Determine the number of characteristics for the generator
        numListLoad1 = int(np.nanmax(loadInfo[:, 1]))
        numListLoad2 = int(np.nanmax(loadInfo[:, loadInfo.shape[1] - 2]))
        numListLoad3 = max(numListLoad1, numListLoad2)

        # Separate the information of each load
        data_load = data_load.values[:, idLoadInfo==False]
        idx = np.where(pd.isnull(data_load))[0][0]
        data_load = data_load[idx+1:, :]
        idx = np.where(pd.isnull(data_load[:, 0]) == False)[0]
        data_load = data_load[idx, :]

        # Eliminate the NaN elements
        idNanElement = pd.isnull(loadInfo[:, 0])
        loadInfo = loadInfo[pd.notna(loadInfo[:, 1]), :]
        loadInfo = loadInfo[:, [0, 1, 3]]

        # idNanElement equals to 0 when there is a value
        loadType = loadText[np.where(idNanElement == 0)[0]+1, 3]
        temp_res = self._convertStrFlag(loadType, 6, 0, loadOpt, loadOptName)
        loadInfo[np.where(idNanElement == False)[0] -2, 2] = temp_res

        numLoad = int(max(loadInfo[:, 0]))
        idLoad = np.arange(numLoad, dtype=int)
        numListLoad = int((data_load.shape[0]) / numLoad)

        # Define the dictionary to pass the values
        load = {}
        temp_limit = np.zeros((numLoad, data_load.shape[1], numListLoad))
        for i in np.arange(numListLoad):
            temp_limit[:, :, i] = data_load[idLoad * numListLoad + i, :]
        load['limit'] = temp_limit

        # Owner dictionary
        self.owner['load'] = np.array(loadInfo[idLoad * numListLoad1 + 2, 2], dtype=int)

        # InfoLoad
        infoLoad = loadInfo[:, 2]
        infoLoad = np.reshape(infoLoad, (numListLoad3, numLoad), order='F')
        load['info'] = infoLoad

        # Assign the consumer type
        load['typeConsumer'] = loadType

        # Consumer Location set
        loadInBus = np.zeros((int(max(self.pD_network['busNum'][0])), numLoad))
        idx = np.where(pd.notna(loadInfo[:, 0]))[0]
        idLoadLoc = np.array(loadInfo[idx, 2], dtype=int)

        idLoadLoc = np.ravel_multi_index([idLoadLoc.transpose(), idLoad],
                                          dims=(loadInBus.shape[0], loadInBus.shape[1]),
                                          order='F') - 1

        temp_loadInBus = loadInBus.ravel()
        temp_loadInBus[idLoadLoc] = 1
        temp_loadInBus = np.reshape(temp_loadInBus,
                                    (loadInBus.shape[0], loadInBus.shape[1]),
                                    order='F')

        self.load = load
        self.resources['loadInBus'] = temp_loadInBus
        self.resources['numLoad'] = numLoad

        return


    def _storOption(self):

        options = None
        names = None
        if self.vals is not None:
            options = self.vals
        else:
            options = [1, 2, 3, 4,
                       1, 2, 3, 4, 5]

            names = ['Li-ion', 'Lead acid', 'Flow', 'Na-NiC12', 'Cap',
                     'Dch Max', 'Ch Max', 'Dch Cap', 'Ch Cap']
        return options, names


    def read_storage(self):

        # Read the Excel
        sheet_name_stor = 'Storage_EC{}'.format(self.ecID)
        data_storage = pd.read_excel(self.path, sheet_name=sheet_name_stor, header=None)

        # Conversion from storageData
        storOpt, storOptName = self._storOption()

        # Copy and turn the DataFrame to numeric
        data_storNumeric = data_storage.copy(deep=True)
        idSimPer = np.where(data_storNumeric.to_numpy()[0, :] == self.resources['period'])[0][0]
        data_storNumeric.drop(data_storNumeric.columns[idSimPer+1:], inplace=True)

        data_storNumeric = data_storNumeric.to_numpy()
        for i in np.arange(data_storNumeric.shape[0]):
            data_storNumeric[i, :] = pd.to_numeric(data_storNumeric[i, :], errors='coerce')

        # Find the id number of storage and bus locations
        idStorInfo = pd.isnull(data_storNumeric[0, :])
        storInfo = data_storNumeric[:, idStorInfo]

        # Number of characteristics for the storage
        numListStor1 = int(np.nanmax(storInfo[:, 1]))
        numListStor2 = int(np.nanmax(storInfo[:, storInfo.shape[1] - 2]))
        numListStor3 = max(numListStor1, numListStor2)

        # Separate the information of each storage
        data_storNumeric = data_storNumeric[:, idStorInfo == False]
        idx = np.where(pd.isnull(data_storNumeric[:, 0]))[0]
        data_storNumeric = data_storNumeric[idx[0]:, :]
        idx = np.where(pd.notna(data_storNumeric[:, 0]))[0]
        data_storNumeric = data_storNumeric[idx, :]

        # Eliminate the NaN elements
        idNanElement = pd.isnull(storInfo[:, 0])
        storInfo = pd.DataFrame(storInfo)
        storInfo.drop(np.where(pd.isnull(storInfo.to_numpy()[:, 1]))[0], axis=0, inplace=True)
        storInfo = storInfo.to_numpy()
        storInfo = storInfo[:, [0, 1, 3]]

        storType = data_storage.to_numpy()[np.where(idNanElement == False)[0] + 1, 3]
        temp_stor = self._convertStrFlag(storType, 3, 0, storOpt, storOptName)
        storInfo[np.where(idNanElement == False)[0] - 2, 2] = temp_stor

        # Determine the number of storage
        numStor = int(np.nanmax(storInfo[:, 0]))
        idStor = np.arange(numStor, dtype=int)
        numListStor2 = int(data_storNumeric.shape[0] / numStor)

        temp_limit = np.zeros((numStor, data_storNumeric.shape[1], numListStor2))
        for i in np.arange(numListStor2):
            temp_limit[:, :, i] = data_storNumeric[idStor * numListStor2 + i, :]

        self.owner['stor'] = storInfo[idStor * numListStor1 + 2, 2]

        # Pass the data into a dictionary
        infoStor = storInfo[:, 2]
        idx = pd.isnull(infoStor)
        infoStor[idx] = 0
        infoStor = np.reshape(infoStor,
                              (numListStor3, numStor),
                              order='F')

        storage = {'limit': temp_limit,
                   'info': infoStor.transpose(),
                   'typeBattery': storType}

        self.storage = storage

        # Battery Location set
        storInBus = np.zeros((np.nanmax(self.pD_network['busNum']), numStor))
        idx = np.where(pd.notna(storInfo[:, 0]))[0]
        idStorLoc = np.array(storInfo[idx, 2], dtype=int)

        idStorLoc = np.ravel_multi_index([idStorLoc.transpose(), idStor],
                                          dims=(storInBus.shape[0], storInBus.shape[1]),
                                          order='F') - 1


        temp_storBus = storInBus.ravel()
        temp_storBus[idStorLoc] = 1
        temp_storBus = np.reshape(temp_storBus,
                                  (storInBus.shape[0], storInBus.shape[1]),
                                  order='F')
        self.parameterData['resources']['storInBus'] = temp_storBus
        self.parameterData['resources']['numStor'] = numStor

        return


    def read_csData(self):

        # Read the Excel
        sheet_name_cstation = 'CStation_EC{}'.format(self.ecID)
        data_cstation = pd.read_excel(self.path, sheet_name=sheet_name_cstation, header=None)

        # Find the column with the simulationPeriod
        idSimPer = np.where(data_cstation.to_numpy()[0, :] == self.resources['period'])[0][0]
        data_cstation.drop(data_cstation.columns[idSimPer+1:], inplace=True)

        # Find the id number of charging stations of bus locations
        cs_numeric = data_cstation.copy(deep=True).to_numpy()
        for i in np.arange(cs_numeric.shape[0]):
            cs_numeric[i, :] = pd.to_numeric(cs_numeric[i, :], errors='coerce')
        idCStatInfo = pd.isnull(cs_numeric[0, :])
        cstatInfo = cs_numeric[:, idCStatInfo]

        # Determine the number of characteristics for the charging stations
        numListCStat1 = int(np.nanmax(cstatInfo[:, 1]))
        numListCStat2 = int(np.nanmax(cstatInfo[:, cstatInfo.shape[1]-2]))
        numListCStat3 = max(numListCStat1, numListCStat2)

        # Separate the information of each charging station
        cs_numeric = cs_numeric[:, idCStatInfo == False]
        idx = np.where(pd.isnull(cs_numeric[:, 0]))[0]
        cs_numeric = cs_numeric[idx[0] + 1:, :]
        idx = np.where(pd.notna(cs_numeric[:, 0]))[0]
        cs_numeric = cs_numeric[idx, :]

        # Eliminate the NaN elements
        idNanElement = pd.isnull(cstatInfo[:, 0])
        cstatInfo = pd.DataFrame(cstatInfo)
        #cstatInfo = cstatInfo.transpose()
        cstatInfo.drop(np.where(pd.isnull(cstatInfo.to_numpy()[:, 1]))[0],
                       axis=0, inplace=True)
        cstatInfo = cstatInfo.to_numpy()
        cstatInfo = cstatInfo[:, [0, 1, 3]]

        # Determine the number of charging stations
        numCStat = int(np.nanmax(cstatInfo[:, 0]))
        idCStat = np.arange(numCStat, dtype=int)
        numListCStat2 = int(cs_numeric.shape[0] / numCStat)

        # Pass to dictionary
        infoCStat = cstatInfo[:, 2]
        idx = pd.isnull(infoCStat)
        infoCStat[idx] = 0
        infoCStat = np.reshape(infoCStat,
                               (numListCStat3, numCStat),
                               order='F')

        temp_limit = np.zeros((numCStat, cs_numeric.shape[1], numListCStat2))
        for i in np.arange(numListCStat2):
            temp_limit[:, :, i] = cs_numeric[idCStat * numListCStat2 + i, :]

        CS_Place = np.zeros((numCStat, numCStat))
        for i in np.arange(numCStat):
            temp1 = int(infoCStat.transpose()[i, 9] - 1)
            temp2 = int(infoCStat.transpose()[i, 10] - 1)
            CS_Place[i, [temp1, temp2]] = 1

        cstation = {}
        cstation['info'] = infoCStat.transpose()
        cstation['CS_Place'] = CS_Place

        # Charging station location set
        cstatInBus = np.zeros((np.nanmax(self.pD_network['busNum']), numCStat))
        idx = np.where(pd.notna(cstatInfo[:, 0]))[0]
        idCStatLoc = np.array(cstatInfo[idx, 2], dtype=int)

        idCStatLoc = np.ravel_multi_index([idCStatLoc.transpose(), idCStat],
                                           dims=(cstatInBus.shape[0], cstatInBus.shape[1]),
                                          order='F') - 1

        temp_cs = cstatInBus.ravel()
        temp_cs[idCStatLoc] = 1
        temp_cs = np.reshape(temp_cs, (cstatInBus.shape[0], cstatInBus.shape[1]),
                             order='F')

        self.cstation = cstation
        self.parameterData['resources']['cstatInBus'] = temp_cs
        self.parameterData['resources']['numCStat'] = numCStat
        return


    def _peerOption(self):

        options = None
        names = None
        if self.vals is not None:
            options = self.vals
        else:
            options = [1, 2, 3, 4, 5, 6, 7,
                       1, 2, 3, 4, 5, 6]

            names = ['EC Manager', 'Community Member', 'Shared Resource Manager',
                     'Community Manager', 'Flexibility Operator', 'DSO', 'TSO',
                     'Load Max', 'Red Max', 'Cut Max', 'Mov Max', 'In Mov Max',
                     'ENS Max']
        return options, names


    def read_peers(self):

        # Read the Excel sheet
        sheet_name_peers = 'Peers_Info_EC{}'.format(self.ecID)
        data_peers = pd.read_excel(self.path, sheet_name=sheet_name_peers, header=None)

        # Conversion from peersData
        peersOpt, peersOptName = self._peerOption()

        # Find the column with the simulation period
        idSimPer = np.where(data_peers.to_numpy()[0, :] == self.resources['period'])[0][0]

        # Copy and turn the DataFrame to a numpy matrix
        peers_numeric = data_peers.copy(deep=True)
        peers_numeric.drop(peers_numeric.columns[idSimPer+1:], inplace=True)
        peers_numeric = peers_numeric.to_numpy()
        for i in np.arange(peers_numeric.shape[0]):
            peers_numeric[i, :] = pd.to_numeric(peers_numeric[i, :], errors='coerce')

        # Find the ID number of peers and bus location
        idPeersInfo = pd.isnull(peers_numeric[0, :])
        peersInfo = peers_numeric[:, idPeersInfo]

        # Determine the number of characteristics for the peers
        numListPeers1 = int(np.nanmax(peersInfo[:, 1]))
        numListPeers2 = int(np.nanmax(peersInfo[:, peersInfo.shape[1]-2]))
        numListPeers3 = max(numListPeers1, numListPeers2)

        # Separate the information of each peer
        peers_numeric = peers_numeric[:, idPeersInfo == False]
        idx = np.where(pd.isnull(peers_numeric[:, 0]))[0]
        peers_numeric = peers_numeric[idx[0]+1:, :]
        idx = np.where(pd.notna(peers_numeric[:, 0]))[0]
        peers_numeric = peers_numeric[idx, :]

        # Eliminate the NaN elements
        idNanElement = pd.isnull(peersInfo[:, 0])
        peersInfo = pd.DataFrame(peersInfo)
        peersInfo.drop(np.where(pd.isnull(peersInfo.to_numpy()[:, 1]))[0], axis=0, inplace=True)
        peersInfo = peersInfo.to_numpy()
        peersInfo = peersInfo[:, [0, 1, 3]]
        peersType = data_peers.to_numpy()[np.where(idNanElement == False)[0], 3]

        temp_peers = self._convertStrFlag(peersType, 6, 0, peersOpt, peersOptName)
        peersInfo[np.where(idNanElement == False)[0] - 3, 2] = temp_peers

        # Determine the number of peers
        numPeers = int(np.nanmax(peersInfo[:, 0]))
        idPeers = np.arange(numPeers, dtype=int)
        numListPeers2 = int(peers_numeric.shape[0] / numPeers)

        # Pass the values
        temp_limit = np.zeros((numPeers, peers_numeric.shape[1], numListPeers2))
        for i in np.arange(numListPeers2):
            temp_limit[:, :, i] = peers_numeric[idPeers * numListPeers2 + i, :]

        # Pass to the dictionary
        infoPeers = peersInfo[:, 2]
        infoPeers = np.reshape(infoPeers, (numListPeers3, numPeers),
                               order='F')

        # Assign the values
        peers = {'info': infoPeers.transpose(),
                 'typeRole': peersType}
        self.peers = peers

        self.parameterData['resources']['numPeers'] = numPeers
        return


    def readExcel(self):

        # Excel sheets to read from the Excel
        sheet_name_branch = 'Network_Info'
        sheet_name_peers = 'Peers_Info_EC{}'.format(self.ecID)
        sheet_name_gen = 'Generator_EC{}'.format(self.ecID)
        sheet_name_load = 'Load_EC{}'.format(self.ecID)
        sheet_name_stor = 'Storage_EC{}'.format(self.ecID)
        sheet_name_cstation = 'CStation_EC{}'.format(self.ecID)
        sheet_name_v2g = 'Vehicle_EC{}'.format(self.ecID)

        #data_peers = pd.read_excel(path, sheet_name=sheet_name_peers, header=None)
        #data_gen = pd.read_excel(path, sheet_name=sheet_name_gen, header=None)
        #data_load = pd.read_excel(path, sheet_name=sheet_name_load, header=None)
        #data_storage = pd.read_excel(path, sheet_name=sheet_name_stor, header=None)
        #data_cstation = pd.read_excel(path, sheet_name=sheet_name_cstation, header=None)
        #data_v2g = pd.read_excel(path, sheet_name=sheet_name_v2g, header=None)

        return



data = ExcelReader(path='src/EC_OnlyMembers_V8/EC_V4.xlsx', ecID=1)

In [ ]:
data.network['busRef']